In [18]:
import torch
import matplotlib.pyplot as plt

In [5]:
torch.arange(2, 5, 1).unique()

tensor([2, 3, 4])

In [150]:
sigma_map = torch.randn(1, 2, 100, 100)
sigma_b = sigma_map[0]
instance_labels = torch.zeros((3, 100, 100), dtype=torch.long)
instance_labels[0, 50:60, 50:60] = 1
instance_labels[1, 20:40, 12:39] = 1
instance_labels[2, 79:82, 80:95] = 1
#instance_labels = instance_labels.type(torch.int)

In [151]:
s1 = (sigma_b[:,50:60, 50:60] - sigma_b[:,50:60, 50:60].mean(dim=(1,2,)).unsqueeze(-1).unsqueeze(-1)).pow(2).sum(dim=0).mean()

In [152]:
s2 = (sigma_b[:,20:40, 12:39] - sigma_b[:,20:40, 12:39].mean(dim=(1,2,)).unsqueeze(-1).unsqueeze(-1)).pow(2).sum(dim=0).mean()

In [153]:
s3 = (sigma_b[:,79:82, 80:95] - sigma_b[:,79:82, 80:95].mean(dim=(1,2,)).unsqueeze(-1).unsqueeze(-1)).pow(2).sum(dim=0).mean()

In [231]:
(s1 + s2 + s3)

tensor(5.9851)

In [155]:
sigma_b.shape

torch.Size([2, 100, 100])

In [241]:
s = sigma_b.view(2, -1)
ils = instance_labels.view(3, -1)

mean_ss = torch.zeros(3, 2)

for i in range(3):
    mean_ss[i] = s[:, ils[i]==1].mean(dim=1)

In [245]:
mean_ss.shape

torch.Size([3, 2])

In [229]:
l = 0
for i in range(3):
    l += (s[:,ils[i]==1] - mean_ss[i].unsqueeze(-1)).pow(2).sum(dim=0).mean()

In [230]:
l

tensor(5.9851)

In [184]:
mean_ss[0].shape

torch.Size([3])

In [232]:
def sigma_means_and_smooth_loss(sigma_map, instance_labels):
    """
    sigma_map: [1, <dims>] or [S, <dims>]
    instance_labels: [I, <dims>]
    """
    
    I = instance_labels.shape[0]
    S = sigma_map.shape[0]
    device = sigma_map.device
    sigma_means = torch.zeros((I, S), device=device)
    
    smooth_loss = 0
    
    sigma_map = sigma_map.view(S, -1)
    instance_labels = instance_labels.view(I, -1)
    
    for i in range(I):
        s_at_i = sigma_map[:, instance_labels[i]==1]
        sigma_mean = s_at_i.mean(dim=1)
        smooth_loss += (s_at_i - sigma_mean.unsqueeze(-1)).pow(2).sum(dim=0).mean() # sum over number of S dims, mean over
        # number of pixels in the instance
        
        sigma_means[i] = sigma_mean
        
    return sigma_means, smooth_loss
        

In [233]:
sigma_means_and_smooth_loss(sigma_b, instance_labels)

(tensor([[-0.1038, -0.2355],
         [ 0.0538,  0.0062],
         [ 0.2467, -0.0073]]),
 tensor(5.9851))

In [199]:
positions = torch.stack(
            torch.meshgrid(
                *[torch.arange(0, sdi, 1) for sdi in (192, 168)],
                indexing='ij'
            )
        ).type(torch.float32)

In [201]:
positions.shape

torch.Size([2, 192, 168])

In [204]:
positions * torch.Tensor([2, 3]).unsqueeze(-1).unsqueeze(-1)

tensor([[[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  2.,   2.,   2.,  ...,   2.,   2.,   2.],
         [  4.,   4.,   4.,  ...,   4.,   4.,   4.],
         ...,
         [378., 378., 378.,  ..., 378., 378., 378.],
         [380., 380., 380.,  ..., 380., 380., 380.],
         [382., 382., 382.,  ..., 382., 382., 382.]],

        [[  0.,   3.,   6.,  ..., 495., 498., 501.],
         [  0.,   3.,   6.,  ..., 495., 498., 501.],
         [  0.,   3.,   6.,  ..., 495., 498., 501.],
         ...,
         [  0.,   3.,   6.,  ..., 495., 498., 501.],
         [  0.,   3.,   6.,  ..., 495., 498., 501.],
         [  0.,   3.,   6.,  ..., 495., 498., 501.]]])

In [195]:
full_embedding = torch.zeros(4, 192, 168)

In [196]:
full_embedding[0:2] = positions

In [197]:
full_embedding

tensor([[[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  1.,   1.,   1.,  ...,   1.,   1.,   1.],
         [  2.,   2.,   2.,  ...,   2.,   2.,   2.],
         ...,
         [189., 189., 189.,  ..., 189., 189., 189.],
         [190., 190., 190.,  ..., 190., 190., 190.],
         [191., 191., 191.,  ..., 191., 191., 191.]],

        [[  0.,   1.,   2.,  ..., 165., 166., 167.],
         [  0.,   1.,   2.,  ..., 165., 166., 167.],
         [  0.,   1.,   2.,  ..., 165., 166., 167.],
         ...,
         [  0.,   1.,   2.,  ..., 165., 166., 167.],
         [  0.,   1.,   2.,  ..., 165., 166., 167.],
         [  0.,   1.,   2.,  ..., 165., 166., 167.]],

        [[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

In [246]:
S = 2
I = instance_labels.shape[0]
embeddings = torch.randn(S, 100, 100)
sigma_means = mean_ss
sigma_means.shape, instance_labels.shape

(torch.Size([3, 2]), torch.Size([3, 100, 100]))

In [247]:
e = embeddings.view(S, -1)
e = e.expand(I, S, e.shape[-1])

In [299]:
phi = (-(e - sigma_means.unsqueeze(-1)).pow(2)/sigma_means.pow(2).unsqueeze(-1)).sum(dim=1).exp() + 1

In [300]:
phi.min()

tensor(1.)

In [254]:
e.shape, phi.shape

(torch.Size([3, 2, 10000]), torch.Size([3, 10000]))

In [256]:
from monai.losses import DiceLoss

In [263]:
loss = DiceLoss(sigmoid=True, reduction='sum')

In [265]:
loss(phi.unsqueeze(1), instance_labels.view(I, 1, -1))

tensor(2.8741)

In [308]:
phi.shape

torch.Size([3, 10000])

In [266]:
instance_labels.shape

torch.Size([3, 100, 100])

In [309]:
seed_mask = (phi * instance_labels.view(I, -1)).sum(dim=0)

In [310]:
(seed_mask == 0).sum()

tensor(9315)

In [279]:
(instance_labels==0).sum()

tensor(29315)

In [280]:
instance_labels.unique()

tensor([0, 1])

In [281]:
instance_labels.shape

torch.Size([3, 100, 100])

In [306]:
(instance_labels.sum(0)==0).sum()

tensor(9315)

In [303]:
(phi == 0).sum()

tensor(0)

In [293]:
seed_mask.shape

torch.Size([2, 10000])

In [307]:
instance_labels.shape

torch.Size([3, 100, 100])

In [ ]:
a = torch.randn(*instance_labels.shape)